In [1]:
import pandas as pd
import altair as alt

In [2]:
alt.data_transformers.disable_max_rows()
brush = alt.selection(type='interval')

In [3]:
data = pd.read_csv("../github_code4rena.csv")

In [4]:
data["date"] = pd.to_datetime(data.date)
data["handle"] = pd.Categorical(data.handle)

plt_data = pd.concat([data.groupby("handle")["date"].min().rename('start'), 
                      data.groupby("handle")["date"].max().rename('end')], axis=1, names=['start', 'end']).reset_index()
plt_data.sort_values("end", ascending=False, inplace=True)

In [5]:
gantt = alt.Chart(plt_data, width=800).mark_bar().encode(
    x=alt.X('start:T', axis=alt.Axis(format='%m/%y', title='First and last contest')),
    x2='end:T',
    y=alt.Y('handle:N', sort='x'),
    tooltip=['handle:N', 'start:T', 'end:T']
    #color='handle:N'
).transform_filter(
    brush
).interactive() # Graph unique participants' handles sorted by their first submission and their last submission

In [6]:
hist = alt.Chart(plt_data.drop("end", axis=1), width=800).mark_bar(size=10).encode(
    x=alt.X('start:T', axis=alt.Axis(format='%m/%y', title='Participant\'s first contest')),
    y=alt.Y('count(handle):Q', axis=alt.Axis(title='Number of new participants'))
).add_selection( # Allow filtering the handles in the gantt chart to target specific starting dates
    brush
)

In [7]:
hist & gantt

alt.VConcatChart(...)